In [4]:
from pandas import read_csv
import numpy
import cv2
from PIL import Image

In [5]:
n_amostras = 794
n_sensores = 64
erro_minimo = numpy.float64('1.0e-4')

In [6]:
print("Importando sinal da matriz H")
matriz_H = read_csv(filepath_or_buffer='../files/H-1/H-1.csv', header=None, dtype=float).to_numpy()
print("Matriz H importada")

Importando sinal da matriz H
Matriz H importada


In [7]:
sinal_original = read_csv(filepath_or_buffer='../files/avaliacao/sinal1.csv', header=None, dtype=float, sep='\t',
                          decimal=',').to_numpy()

In [8]:
sinal = numpy.zeros(shape=sinal_original.shape)
sinal_old =sinal_original
for _ in range(1):
    for linha in range(n_sensores):
        for coluna in range(n_amostras):
            epsulon = 100 + 1 / 20 * coluna * numpy.sqrt(coluna)
            array_position = linha * n_amostras + coluna
            sinal[array_position, 0] = sinal_old[array_position, 0] * epsulon
    sinal_old = sinal

In [9]:
f_old = numpy.zeros((3600, 1), dtype=numpy.float64)
r_old = numpy.subtract(sinal, numpy.zeros(shape=(matriz_H.shape[0], 1)))
z_old = numpy.matmul(matriz_H.transpose(), r_old)
p_old = z_old

ultimo_erro = numpy.float64('9999.0')
melhor_imagem = f_old
n_iteracao = 0

In [10]:
for _ in range(400):
    n_iteracao += 1
    w_new = numpy.matmul(matriz_H, p_old)
    alpha = numpy.divide(numpy.power(numpy.linalg.norm(z_old), 2), numpy.power(numpy.linalg.norm(w_new), 2))
    f_new = numpy.add(f_old, alpha * p_old)
    r_new = numpy.subtract(r_old, w_new * alpha)
    z_new = numpy.matmul(matriz_H.transpose(), r_new)
    beta = numpy.divide(numpy.power(numpy.linalg.norm(z_new), 2), numpy.power(numpy.linalg.norm(z_old), 2))
    p_new = numpy.add(z_new, beta * p_old)
    erro = numpy.absolute(numpy.subtract(numpy.linalg.norm(r_new, ord=2), numpy.linalg.norm(r_old, ord=2)))
    if erro < ultimo_erro:
        melhor_imagem = f_new
        ultimo_erro = erro
    if erro < erro_minimo:
        print("Bateu no erro minimo")
        break
    f_old = f_new
    r_old = r_new
    z_old = z_new
    p_old = p_new

Bateu no erro minimo


In [11]:
reshape = melhor_imagem.reshape(60, 60)

In [12]:
normalized = cv2.normalize(src=reshape, alpha=0, beta=255, dst=numpy.zeros_like(reshape), norm_type=cv2.NORM_MINMAX)

In [13]:
first_image = Image.fromarray(numpy.uint8(normalized.transpose()), mode='L')

In [14]:
first_image.save(fp='images/first.bmp')